In [1]:
import numpy as np
import pandas as pd
from keras.layers import Dense, Input, LSTM
import plotly.express as exp
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.models import Model
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
SA_train = pd.read_csv('/content/SentimentAnalysisTrain.csv', encoding = 'latin')
SA_test = pd.read_csv('/content/SentimentAnalysisTest.csv', encoding = 'latin')

In [4]:
SA_train.head()

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [5]:
SA_train = SA_train[['text', 'sentiment']]
SA_test = SA_test[['text', 'sentiment']]

In [6]:
import re

In [8]:
for i in [SA_train, SA_test]:
    i['text'] = i['text'].apply((lambda x: (re.sub('[^a-zA-z0-9\s]','', str(x).lower()).strip().split(' '))))

In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding

In [21]:
tokenizer = Tokenizer(num_words=1200)
tokenizer.fit_on_texts(SA_train['text'])

train_sequences = tokenizer.texts_to_sequences(SA_train['text'])
test_sequences = tokenizer.texts_to_sequences(SA_test['text'])

train_sequences = pad_sequences(train_sequences, maxlen=50)
test_sequences = pad_sequences(test_sequences, maxlen=50)

In [22]:
print(train_sequences.shape)
train_sequences

(27481, 50)


array([[  0,   0,   0, ...,  71,   2, 137],
       [  0,   0,   0, ...,   8, 100,   9],
       [  0,   0,   0, ..., 103,  12,  94],
       ...,
       [  0,   0,   0, ...,   1, 178, 689],
       [  0,   0,   0, ...,  23, 721,  92],
       [  0,   0,   0, ...,   4, 506,   1]], dtype=int32)

In [24]:
y_train = pd.get_dummies(SA_train['sentiment']).values
y_test = pd.get_dummies(SA_test['sentiment']).values

In [18]:
from keras.layers import SpatialDropout1D

In [25]:
inputs = Input(train_sequences.shape[1])
x = Embedding(1200, 99, input_length = train_sequences.shape[1])(inputs)
x = SpatialDropout1D(0.5)(x)
x = LSTM(100)(x)
outputs = Dense(3, activation = 'softmax')(x)
model = Model(inputs, outputs)
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 50)]              0         
                                                                 
 embedding_2 (Embedding)     (None, 50, 99)            118800    
                                                                 
 spatial_dropout1d_2 (Spati  (None, 50, 99)            0         
 alDropout1D)                                                    
                                                                 
 lstm_2 (LSTM)               (None, 100)               80000     
                                                                 
 dense_2 (Dense)             (None, 3)                 303       
                                                                 
Total params: 199103 (777.75 KB)
Trainable params: 199103 (777.75 KB)
Non-trainable params: 0 (0.00 Byte)
___________________

In [26]:
model.fit(train_sequences, y_train, epochs = 10, validation_split=0.2)

Epoch 1/10
687/687 [==============================] - 32s 43ms/step - loss: 0.8976 - accuracy: 0.5762 - val_loss: 0.7846 - val_accuracy: 0.6593
Epoch 2/10
687/687 [==============================] - 29s 43ms/step - loss: 0.7621 - accuracy: 0.6689 - val_loss: 0.7641 - val_accuracy: 0.6665
Epoch 3/10
687/687 [==============================] - 30s 44ms/step - loss: 0.7323 - accuracy: 0.6855 - val_loss: 0.7569 - val_accuracy: 0.6729
Epoch 4/10
687/687 [==============================] - 31s 45ms/step - loss: 0.7118 - accuracy: 0.6952 - val_loss: 0.7421 - val_accuracy: 0.6838
Epoch 5/10
687/687 [==============================] - 30s 43ms/step - loss: 0.6945 - accuracy: 0.7022 - val_loss: 0.7427 - val_accuracy: 0.6838
Epoch 6/10
687/687 [==============================] - 30s 43ms/step - loss: 0.6783 - accuracy: 0.7112 - val_loss: 0.7456 - val_accuracy: 0.6851
Epoch 7/10
687/687 [==============================] - 29s 43ms/step - loss: 0.6603 - accuracy: 0.7179 - val_loss: 0.7633 - val_accuracy:

In [2]:
electricity = pd.read_csv('/content/Electricity_load_forecasting - Electricity_load_forecasting.csv')

In [3]:
electricity.head()

,datetime,nat_demand,T2M_toc,QV2M_toc,TQL_toc,W2M_toc,T2M_san,QV2M_san,TQL_san,W2M_san,T2M_dav,QV2M_dav,TQL_dav,W2M_dav,Holiday_ID,holiday,school
0,03-01-2015 1:00,970.3450,25.865259,0.018576,0.016174,21.850546,23.482446,0.017272,0.001855,10.328949,22.662134,0.016562,0.096100,5.364148,0,0,0
1,03-01-2015 2:00,912.1755,25.899255,0.018653,0.016418,22.166944,23.399255,0.017265,0.001327,10.681517,22.578943,0.016509,0.087646,5.572471,0,0,0
2,03-01-2015 3:00,900.2688,25.937280,0.018768,0.015480,22.454911,23.343530,0.017211,0.001428,10.874924,22.531030,0.016479,0.078735,5.871184,0,0,0
3,03-01-2015 4:00,889.9538,25.957544,0.018890,0.016273,22.110481,23.238794,0.017128,0.002599,10.518620,22.512231,0.016487,0.068390,5.883621,0,0,0
4,03-01-2015 5:00,893.6865,25.973840,0.018981,0.017281,21.186089,23.075403,0.017059,0.001729,9.733589,22.481653,0.016456,0.064362,5.611724,0,0,0


In [4]:
electricity = electricity.drop(columns = ['Holiday_ID'])

In [5]:
rolling_window = electricity['nat_demand'].rolling(window=700, min_periods = 1)
electricity['nat_demand_ma'] = rolling_window.mean()

In [6]:
electricity['nat_demand_lag'] = electricity['nat_demand'].shift(1)

In [7]:
electricity = electricity.set_index('datetime')

In [8]:
electricity = electricity.dropna()

In [9]:
x = electricity.drop(columns = ['nat_demand'])
y = electricity['nat_demand']

In [10]:
x_train, y_train = x.iloc[:35020], y.iloc[:35020]
x_test, y_test = x.iloc[35020:], y.iloc[35020:]

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train.iloc[:,:] = scaler.fit_transform(x_train)
x_test.iloc[:,:] = scaler.transform(x_test)

<ipython-input-11-cf368fa7a3a1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train.iloc[:,:] = scaler.fit_transform(x_train)
<ipython-input-11-cf368fa7a3a1>:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  x_train.iloc[:,:] = scaler.fit_transform(x_train)
<ipython-input-11-cf368fa7a3a1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [12]:
def timestep(df, size):
    X, y = [], []
    num_of_features = df.shape[1] - 1
    for i in range(size, df.shape[0]):
        X.append(df.iloc[i-size:i, 1:num_of_features+1].values)
        y.append(df.iloc[i-size:i, 0])
    X, y = np.array(X), np.array(y)
    X = X.reshape(df.shape[0]-size, size, num_of_features)

    return X, y

In [13]:
from sklearn.preprocessing import StandardScaler

In [14]:
train, test = train_test_split(electricity, test_size=0.2, shuffle=False)

In [15]:
#Scaling
scaler = StandardScaler()
train.iloc[:, 1:] = scaler.fit_transform(train.iloc[:, 1:])
test.iloc[:, 1:] = scaler.transform(test.iloc[:, 1:])

In [31]:
X_train, y_train = timestep(train, 4)
X_test, y_test = timestep(test, 4)

In [32]:
input = Input((X_train.shape[1:]))
x = LSTM(100, return_sequences=True)(input)
x = LSTM(100)(x)
x = Dense(16, activation='relu')(x)
x = Dense(8, activation='relu')(x)
output = Dense(1, activation='linear')(x)
model = Model(input,output)

In [33]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 4, 16)]           0         
                                                                 
 lstm_4 (LSTM)               (None, 4, 100)            46800     
                                                                 
 lstm_5 (LSTM)               (None, 100)               80400     
                                                                 
 dense_6 (Dense)             (None, 16)                1616      
                                                                 
 dense_7 (Dense)             (None, 8)                 136       
                                                                 
 dense_8 (Dense)             (None, 1)                 9         
                                                                 
Total params: 128961 (503.75 KB)
Trainable params: 128961 (

In [34]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
model.fit(X_train, y_train, epochs=25, validation_split = 0.2)

Epoch 1/25
876/876 [==============================] - 15s 12ms/step - loss: 484566.5312 - mae: 548.7530 - val_loss: 49556.8750 - val_mae: 171.6681
Epoch 2/25
876/876 [==============================] - 8s 9ms/step - loss: 17738.4434 - mae: 98.0510 - val_loss: 4321.7866 - val_mae: 48.9131
Epoch 3/25
876/876 [==============================] - 9s 11ms/step - loss: 3942.2971 - mae: 44.8457 - val_loss: 3830.1348 - val_mae: 45.2399
Epoch 4/25
876/876 [==============================] - 9s 11ms/step - loss: 3851.3997 - mae: 44.0466 - val_loss: 3746.0090 - val_mae: 44.5477
Epoch 5/25
876/876 [==============================] - 8s 9ms/step - loss: 3827.9397 - mae: 43.8481 - val_loss: 3767.9160 - val_mae: 44.4729
Epoch 6/25
876/876 [==============================] - 9s 11ms/step - loss: 3821.0789 - mae: 43.8158 - val_loss: 3732.3723 - val_mae: 44.2317
Epoch 7/25
876/876 [==============================] - 10s 12ms/step - loss: 3812.6121 - mae: 43.7565 - val_loss: 3726.4285 - val_mae: 44.3579
Epoch 8

In [36]:
y_pred = model.predict(X_test)
pred_df = pd.DataFrame({'datetime': test.index[:-4], 'pred': y_pred.reshape(-1)})

274/274 [==============================] - 2s 6ms/step


In [37]:
X_test.shape

(8751, 4, 16)

In [38]:
rmse = mean_squared_error(test.nat_demand[:-4], y_pred)**0.5
mae = mean_absolute_error(test.nat_demand[:-4], y_pred)
r2 = r2_score(test.nat_demand[:-4], y_pred)
print('RMSE:',rmse)
print('MAE:',mae)
print('R2 Score:',r2)

RMSE: 79.68305986783587
MAE: 62.368328484278145
R2 Score: 0.8224741716480002
